#Space Shuttle O-Ring Failures - Logistic Regression Example

## Setup

In [ ]:
##Clear the environment
rm(list=ls())

##Turn off scientific notations for numbers
options(scipen = 999)  

##Set locale
Sys.setlocale("LC_ALL", "English") 

##Set seed for reproducibility
set.seed(2345)

# Turn off warnings
options(warn = -1)

getstats <- function(cm){
  # Sensititvity a.k.a TPR
  tpr <-cm[2,2]/(cm[2,2]+cm[2,1])
  fpr <-cm[1,2]/(cm[1,2]+cm[1,1])
  
  # Specificity a.k.a. TNR
  tnr <- cm[1,1]/(cm[1,1]+cm[1,2])
  fnr <- cm[2,1]/(cm[2,1]+cm[2,2])
  
  # Calculate accuracy
  acc <-(cm[2,2]+cm[1,1])/sum(cm)
  err <-(cm[1,2]+cm[2,1])/sum(cm)
  
  #Precision - Positive Predictive Value
  ppv <- cm[2,2]/(cm[2,2]+cm[1,2])
  
  # Negative Predictive Value
  npv <- cm[1,1]/(cm[1,1]+cm[2,1])
  
  rbind(TruePos_Sensitivity=tpr, FalsePos=fpr, TrueNeg_Specificty=tnr, FalseNeg=fnr, PositivePredictiveValue=ppv, NegativePredictiveValue=npv, Accuracy = acc, Error = err)
}

# clean the data names and data
# Use: df<-cleanit(df)
cleanit <-function(df){
  names(df) <-tolower(names(df))
  names(df) <- gsub("\\(","",names(df))
  names(df) <- gsub("\\)","",names(df))
  names(df) <- gsub("\\.","",names(df))
  names(df) <- gsub("_","",names(df))
  names(df) <- gsub("-","",names(df))
  names(df) <- gsub(",","",names(df))
  return(df)
}


thres=.3 # Start at 50%
nbrclust = 2

##Data

On January 27, 1986, the night before the space shuttle Challenger exploded, engineers at the company that built the shuttle warned National Aeronautics and Space Administration (NASA) scientists that the shuttle should not be launched because of predicted cold weather. Fuel seal problems, which had been encountered in earlier flights, were suspected of being associated with low temperatures. It was argued, that the evidence was inconclusive. The decision was made to launch, even though the temperature at launch time was 29 F.

The data consists of temperatures (degrees Fahrenheit) and an indicator of O-ring failures for 24 space shuttle launches prior to the Challenger disaster. The data in this example comes from Kelly, D. L., & Smith, C. L., (2008). “Risk Analysis of the Space Shuttle: Pre-Challenger Bayesian Prediction of Failure,” NASA Space Systems Engineering & Risk Management Symposium https://inldigitallibrary.inl.gov/sti/3901032.pdf

In [ ]:
##Build dataframe
flt<-c('1','2','3','4','5','6','7','8','9','41-B','41-C','41-D','41-G','51-A','51-C','51-D','51-B','51-G','51-F','51-I','51-J','61-A','61-B','61-C')
temp<-c(66,70,69,80,68,67,72,73,70,57,63,70,78,67,53,67,75,70,81,76,79,75,76,58)
damage<-c(0,1,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0,0,1,0,1)
df<-data.frame(flt,temp,damage)
rm(temp)
rm(damage)
rm(flt)
str(df)

In [ ]:
summary(df)
sapply(df, mean)
sapply(df,sd)

## two-way contingency table of categorical outcome and predictors
xtabs(~damage + temp, data = df)

## Correlation Matrix for numeric variables
cor(df[,c(2:3)])

###View the Data

In [ ]:
par(mfrow=c(2,2))
hist(df$temp)
boxplot(df$temp)
hist(df$damage)
boxplot(df$damage)

In [ ]:
#Enhanced Scatter plot
# Add boxplots to a scatterplot
par(fig=c(0,0.8,0,0.8), new=TRUE)
plot(df$temp, df$damage, xlab="Temp",  ylab="Failure")
#X Variable
par(fig=c(0,0.8,0.55,1), new=TRUE)
boxplot(df$temp, horizontal=TRUE, axes=FALSE)
# Y Variable
par(fig=c(0.65,1,0,0.8),new=TRUE)
boxplot(df$damage, axes=FALSE)
mtext("Enhanced Scatterplot", side=3, outer=TRUE, line=-3)

##Establish Baseline

In [ ]:
# Baseline on Training data 
# Determine the Majority
bl <-table(df$damage)
majority<-ifelse(bl[1]>bl[2],0,1)

# Fill in a prediction for the majority
predictTrainBase <-rep(majority,nrow(df))
#Compare
cm <- table(df$damage,predictTrainBase, exclude=NULL)
addmargins(cm)
getstats(cm)

In [ ]:
# Baseline based on Majority a.k.a. Naive Baseline
# Determine the Majority
#table(df$damage)
majority<- ifelse(table(df$damage)[[1]]>table(df$damage)[[2]], 0, 1)

# Fill in a prediction
predictBase <-rep(majority,nrow(df))

#Compare
table(df$damage,predictBase, exclude=NULL)
cm <- table(df$damage,predictBase, exclude=NULL)
addmargins(cm)
getstats(cm)

###Pull Out Errors

#### False Positives

In [ ]:
# Pull out mistakes
subset(df, predictBase >= thres & damage == 0)

####False Negatives

In [ ]:
subset(df,  predictBase <= thres & damage == 1)

##Build a Logistic Regression Model

In [ ]:
# Build the model Dep and Independent Vars define columns we will be working with
depvar <- 'damage'
indepvars <-c('temp')
vars<- paste(depvar,paste(indepvars,collapse=' + '),sep=' ~ ')

In [ ]:
#Fit the simple model
fit01<-glm(vars,data=df,family=binomial(logit))

###Output

In [ ]:
#Review Output
summary(fit01) # display results

In the output above, the first thing we see is the call which is reminding us what the model we ran was, what options we specified, etc.

Next we see the deviance residuals, which are a measure of model fit. This part of output shows the distribution of the deviance residuals for individual cases used in the model. 

The next part of the output shows the coefficients, their standard errors, the z-statistic (sometimes called a Wald z-statistic), and the associated p-values.  The logistic regression coefficients give the change in the log odds of the outcome for a one change in the predictor variable.  
    
    In this example, we see temp is statistically significant. 
    For every one unit increase in temp, the log odds of failure (versus non-failure) decreases by 0.2360

Below the table of coefficients are fit indices, including the null and deviance residuals and the AIC. 

###Confidence Intervals

We can use the confint function to obtain confidence intervals for the coefficient estimates. Note that for logistic models, confidence intervals are based on the profiled log-likelihood function. 

In [ ]:
## CIs using profiled log-likelihood
# 95% CI for the coefficients
# If the confidence interval contains 1, the results are not statisticlly significant
confint(fit01) 

We can also get CIs based on just the standard errors by using the default method.

In [ ]:
## CIs using standard errors
# 95% CI for the coefficients
# If the confidence interval contains 1, the results are not statisticlly significant
confint.default(fit01)

To put it all in one table, we use cbind to bind the coefficients and confidence intervals column-wise.

In [ ]:
cbind(Log_Like = coef(fit01), confint(fit01))

In [ ]:
cbind(Log_Like = coef(fit01), confint.default(fit01))

###Odds Ratio

Odds ratios are used to compare the relative odds of the occurrence of the outcome of interest (e.g. disease or disorder), given exposure to the variable of interest (e.g. health characteristic, aspect of medical history). The odds ratio can also be used to determine whether a particular exposure is a risk factor for a particular outcome, and to compare the magnitude of various risk factors for that outcome.

    OR=1 Exposure does not affect odds of outcome
    OR>1 Exposure associated with higher odds of outcome
    OR<1 Exposure associated with lower odds of outcome


We can exponentiate the coefficients and interpret them as odds-ratios. R will do this computation for you. To get the exponentiated coefficients, you tell R that you want to exponentiate (exp), and that the object you want to exponentiate is called coefficients and it is part of fit (coef(fit)).

In [ ]:
exp(coef(fit01)) # exponentiated coefficients

We can use the same logic to get odds ratios and their confidence intervals, by exponentiating the confidence intervals from before. 

In [ ]:
exp(confint(fit01)) # 95% CI for exponentiated coefficients

In [ ]:
exp(confint.default(fit01))# 95% CI for exponentiated coefficients using standard error

To put it all in one table, we use cbind to bind the coefficients and confidence intervals column-wise.

In [ ]:
## odds ratios and 95% CI
exp(cbind(OR = coef(fit01), confint(fit01)))

In [ ]:
## odds ratios and 95% CI
exp(cbind(OR = coef(fit01), confint.default(fit01)))

Now we can say that for a one unit increase in temp, the odds of failure (versus non-failure ) decrease by a factor of .79. Note that while R produces it, the odds ratio for the intercept is not generally interpreted.

##Predictions

In [ ]:
predictTrain<-predict(fit01, type="response") # predicted values
#residuals(fit, type="deviance") # residuals

# Confusion Matrix
#table(fit01$fitted.values>=thres,df$damage)
cm <- table(df$damage,predictTrain>thres)
addmargins(cm)
getstats(cm)

#Get all zeros correct 100% (TRUE NEGATIVE RATE)
#Get 4 out of 7 ones correct 57% (TRUE POSITVE RATE)
#Accuracy got 21 out of 24 correct 87.5%
#Error got 3 out of 24 incorrect 12.5%
#REMEMBER ACCuray is not a good measure of the woth of a model

###Pull Out Errors

####False Positives

In [ ]:
subset(df, predictTrain >= thres & damage == 0)

####False Negatives

In [ ]:
subset(df, predictTrain <= thres & damage == 1)

In [ ]:
# Build Receiver Operator Charastics ROC
library(ROCR)

# Prediction function
ROCRpredTest = prediction(predictTrain,df$damage)

# Performance function
ROCRperfTest = performance(ROCRpredTest, "tpr", "fpr")

# Plot ROC curve and add AUC 
plot(ROCRperfTest, colorize=TRUE, print.cutoffs.at=seq(0,1,by=0.1), text.adj=c(-0.2,1.7))
abline(coef=c(0,1))
auc = as.numeric(performance(ROCRpredTest, "auc")@y.values)
text(0.5, 1, "AUC:")
text(0.6,1, round(auc,4))

### Rule of Thumb Intrepretations

If the area under ROC is:
    
    No discrimination:         0.5  
    Acceptable discrimination: 0.7 <= ROC area < 0.8 
    Excellent discrimination:  0.8 <= ROC area < 0.9 
    Outstanding discrimination ROC area >= 0.9 

##Clusters

### Standardize the variables

In [ ]:
df <- transform (df,temp_scaled=scale(temp) )# standardize temp variable

In [ ]:
str(df)

In [ ]:
fit02 <- kmeans(df$temp_scaled, nbrclust)

# get cluster means 
aggregate(df$temp_scaled,by=list(fit02$cluster),FUN=mean)

# append cluster assignment
mydata <- data.frame(df$temp_scaled, fit02$cluster)

In [ ]:
# Cluster Plot against 1st 2 principal components
library(fpc)
library(cluster) 

# vary parameters for most readable graph
clusplot(df$temp_scaled, fit02$cluster, color=TRUE, shade=TRUE, labels=2, lines=0)

# Centroid Plot against 1st 2 discriminant functions
plotcluster(df$temp_scaled, fit02$cluster)

In [ ]:
plotcluster(df$temp, fit02$cluster)

In [ ]:
# Ward Hierarchical Clustering
d <- dist(df$temp_scaled, method = "euclidean") # distance matrix
fit03 <- hclust(d, method="ward.D2") 

In [ ]:
plot(fit03) # display dendogram
groups <- cutree(fit03, k=nbrclust) # cut tree into 5 clusters
# draw dendogram with red borders around the 5 clusters 
rect.hclust(fit03, k=nbrclust, border="red")

In [ ]:
# Load groups
df$groups <- cutree(fit03, nbrclust) # cut tree 

In [ ]:
aggdata= aggregate(.~ groups, data=df, FUN=mean) # Aggregation by group and computation of the mean values
proptemp=aggregate(damage~ groups, data=df, FUN=length) # Computation of the number of observations by group
aggdata$nbr=proptemp$damage
aggdata$proportion=(proptemp$damage)/sum(proptemp$damage) # Computation of the proportion by group
aggdata=aggdata[order(aggdata$proportion,decreasing=T),] # Ordering from the largest group to the smallest

In [ ]:
aggdata

In [ ]:
df[order(df$group,decreasing=T),]

In [ ]:
df[order(df$temp,decreasing=T),]

In [ ]:
nbrclust = 3